## Task Scheduler with Priorities 

- Console-based scheduler: 

-   Add tasks with priority and deadlines 

-   View pending tasks sorted by priority 

-   Mark tasks as done 

-   Archive completed tasks 

Concepts: Heap queue (heapq), file persistence, classes 

In [3]:
import heapq
from datetime import datetime
import os
import json

pending = "pending_tasks.json"
archive = "archived_tasks.json"

class Task:
    def __init__(self, title, priority, deadline, created_at=None, done=False):
        self.title = title
        self.priority = priority
        self.deadline = deadline  # string: YYYY-MM-DD
        self.created_at = created_at or datetime.now().isoformat()
        self.done = done

    def __lt__(self, other):
        return (self.priority, self.deadline) < (other.priority, other.deadline)

    def __repr__(self):
        return f"[{'X' if self.done else ' '}] {self.title} (Priority: {self.priority}, Deadline: {self.deadline})"

    def to_dict(self):
        return {
            "title": self.title,
            "priority": self.priority,
            "deadline": self.deadline,
            "created_at": self.created_at,
            "done": self.done
        }
    
    @classmethod
    def from_dict(cls, data):
        return cls(**data)

class TaskScheduler:
    def __init__(self):
        self.pending_tasks = self.load_tasks(pending)
        heapq.heapify(self.pending_tasks)
        self.archived_tasks = self.load_tasks(archive)

    def load_tasks(self, filename):
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                data = json.load(f)
                return [Task.from_dict(d) for d in data]
        return []

    def save_tasks(self, filename, tasks):
        with open(filename, 'w') as f:
            json.dump([task.to_dict() for task in tasks], f, indent=2)

    def add_task(self, title, priority, deadline):
        task = Task(title, priority, deadline)
        heapq.heappush(self.pending_tasks, task)
        self.save_tasks(pending, self.pending_tasks)

    def view_tasks(self):
        print("\nPending Tasks:")
        for idx, task in enumerate(sorted(self.pending_tasks)):
            print(f"{idx + 1}. {task}")

    def mark_done(self, index):
        if 0 <= index < len(self.pending_tasks):
            sorted_tasks = sorted(self.pending_tasks)
            task = sorted_tasks.pop(index)
            task.done = True
            self.pending_tasks = sorted_tasks
            heapq.heapify(self.pending_tasks)
            self.archived_tasks.append(task)
            self.save_tasks(pending, self.pending_tasks)
            self.save_tasks(archive, self.archived_tasks)
            print(f"Marked '{task.title}' as done and archived.")
        else:
            print("Invalid task number.")

    def run(self):
        while True:
            print("\nTask Scheduler")
            print("1. Add Task")
            print("2. View Pending Tasks")
            print("3. Mark Task as Done")
            print("4. Exit")
            choice = input("Choose an option: ")

            if choice == "1":
                title = input("Task Title: ")
                try:
                    priority = int(input("Priority (lower # = higher priority): "))
                except ValueError:
                    print("Invalid priority.")
                    continue
                deadline = input("Deadline (YYYY-MM-DD): ")
                self.add_task(title, priority, deadline)

            elif choice == "2":
                self.view_tasks()

            elif choice == "3":
                self.view_tasks()
                try:
                    index = int(input("Enter task number to mark done: ")) - 1
                    self.mark_done(index)
                except ValueError:
                    print("Invalid input.")

            elif choice == "4":
                print("Ending Task Scheduler")
                break
            else:
                print("Invalid option.")


if __name__ == "__main__":
    scheduler = TaskScheduler()
    scheduler.run()


Task Scheduler
1. Add Task
2. View Pending Tasks
3. Mark Task as Done
4. Exit

Task Scheduler
1. Add Task
2. View Pending Tasks
3. Mark Task as Done
4. Exit

Pending Tasks:
1. [ ] Complete Python Assignment 2 (Priority: 1, Deadline: 2025-05-28)

Task Scheduler
1. Add Task
2. View Pending Tasks
3. Mark Task as Done
4. Exit
Ending Task Scheduler


## ATM Machine Simulation 

- PIN-based login 

- View balance, withdraw, deposit, mini statement 

- Validate limits and handle errors 

- Concepts: Classes, exception handling, persistent user data in files 

In [2]:
import json
import os
from datetime import datetime

userData = "users.json"

class User:
    def __init__(self, pin, balance=0.0, statement=None):
        self.pin = pin
        self.balance = balance
        self.statement = statement or []

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("Deposit amount must be positive.")
        self.balance += amount
        self.add_statement(f"Deposited ${amount:.2f}")

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("Withdrawal amount must be positive.")
        if amount > self.balance:
            raise ValueError("Insufficient balance.")
        self.balance -= amount
        self.add_statement(f"Withdrew ${amount:.2f}")

    def add_statement(self, message):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.statement.append(f"{timestamp} - {message}")
        self.statement = self.statement[-5:]

    def view_statement(self):
        print("\nMini Statement:")
        if not self.statement:
            print("No transactions yet.")
        else:
            for entry in self.statement:
                print(entry)

    def to_dict(self):
        return {
            "pin": self.pin,
            "balance": self.balance,
            "statement": self.statement
        }

    @classmethod
    def from_dict(cls, data):
        return cls(data["pin"], data["balance"], data["statement"])


class ATM:
    def __init__(self):
        self.users = self.load_users()

    def load_users(self):
        if os.path.exists(userData):
            with open(userData, 'r') as f:
                data = json.load(f)
                return {pin: User.from_dict(u) for pin, u in data.items()}
        return {}

    def save_users(self):
        with open(userData, 'w') as f:
            data = {pin: user.to_dict() for pin, user in self.users.items()}
            json.dump(data, f, indent=2)

    def login(self, pin):
        if pin not in self.users:
            print("User not found. Creating new account.")
            self.users[pin] = User(pin)
        return self.users[pin]

    def run(self):
        pin = input("Enter your 4-digit PIN: ").strip()
        if not pin.isdigit() or len(pin) != 4:
            print("Invalid PIN format.")
            return

        user = self.login(pin)

        while True:
            print("\nMenu")
            print("1. View Balance")
            print("2. Deposit")
            print("3. Withdraw")
            print("4. Mini Statement")
            print("5. Exit")
            choice = input("Choose an option: ")

            try:
                if choice == "1":
                    print(f"\nCurrent Balance: ${user.balance:.2f}")

                elif choice == "2":
                    amount = float(input("Enter amount to deposit: "))
                    user.deposit(amount)
                    print(f"${amount:.2f} deposited successfully.")

                elif choice == "3":
                    amount = float(input("Enter amount to withdraw: "))
                    user.withdraw(amount)
                    print(f"${amount:.2f} withdrawn successfully.")

                elif choice == "4":
                    user.view_statement()

                elif choice == "5":
                    self.save_users()
                    print("Thank you for using our ATM.")
                    break

                else:
                    print("Invalid option.")

            except ValueError as e:
                print(f"Error: {e}")

            self.save_users()


if __name__ == "__main__":
    atm = ATM()
    atm.run()


User not found. Creating new account.

Menu
1. View Balance
2. Deposit
3. Withdraw
4. Mini Statement
5. Exit
$100.00 deposited successfully.

Menu
1. View Balance
2. Deposit
3. Withdraw
4. Mini Statement
5. Exit
Thank you for using our ATM.


## Event Registration System 

- Create an event with limited slots 

- Allow user registration 

- Send confirmation (print/save to file) 

- Display waitlist 

- Concepts: Queue, classes, file I/O 

In [ ]:
from collections import deque
import uuid # going to use for receipt codes

class Event:
    def __init__(self, title, max_slots):
        self.title = title
        self.max_slots = max_slots
        self.registrations = []
        self.waitlist = deque()
    
    def register_user(self, user_name):
        if user_name in self.registrations or user_name in self.waitlist:
            print(f"{user_name} is already registered/waitlisted.")
            return

        if len(self.registrations) < self.max_slots:
            self.registrations.append(user_name)
            print(f"{user_name} successfully registered for {self.title}.")
            self.save_confirmation(user_name, is_waitlisted=False)
        else:
            self.waitlist.append(user_name)
            print(f"{user_name} added to the waitlist for {self.title}.")
            self.save_confirmation(user_name, is_waitlisted=True)
    
    def save_confirmation(self, user_name, is_waitlisted):
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        unique_id = f"{timestamp}-{uuid.uuid4().hex[:6]}" 
    
        filename = f"confirmation_{user_name.replace(' ', '_')}.txt"
        with open(filename, 'w') as f:
            f.write(f"Event: {self.title}\n")
            f.write(f"User: {user_name}\n")
            f.write(f"Confirmation ID: {unique_id}\n")
            if is_waitlisted:
                f.write("Status: WAITLISTED\n")
            else:
                f.write("Status: CONFIRMED\n")
        print(f"Confirmation saved to {filename} with ID: {unique_id}")

    def show_status(self):
        print("\n--- Registration List ---")
        for idx, user in enumerate(self.registrations, start=1):
            print(f"{idx}. {user}")

        print("\n--- Waitlist ---")
        for idx, user in enumerate(self.waitlist, start=1):
            print(f"{idx}. {user}")
    
def main():
    title = input("Enter event name: ")
    max_slots = int(input("Enter number of slots: "))
    event = Event(title, max_slots)

    while True:
        print("\n1. Register")
        print("2. Show Status")
        print("3. Exit")
        choice = input("Choose an option: ").strip()

        if choice == '1':
            user_name = input("Enter your name: ").strip()
            event.register_user(user_name)
        elif choice == '2':
            event.show_status()
        elif choice == '3':
            print("Exiting.")
            break
        else:
            print("Invalid option.")

if __name__ == "__main__":
    main()


1. Register
2. Show Status
3. Exit
Syed successfully registered for New Concert!
Confirmation saved to confirmation_Syed.txt with ID: 20250528193116-c9a483

1. Register
2. Show Status
3. Exit
Exiting.
